# Start
12月的开始，先给自己定一个整体的目标。这个学期已经快结束了，感觉自己的进度有限，这是一件很折磨人的事情，自己在外漂流忍受艰辛，总要留下什么吧。目前确定自己要做 **SAM** 相关工作，这是一件值得开心的事情，明确了自己要努力的方向，以及这次感觉自己对[`热点`]的感知应该是比较准确的。在拥有一篇论文的情况下，其实是可以申请博士的，但是学校应该是不太能令人如意的。

**ICML 2025 1月 --- ICCV 2025 3月 --- NeurIPS 2025 5月 --- AAAI 2026 8月中**    
离梦想最近的一次,这一次的顶会必须要拿下!!! 希望到月底可以开始做实验。    
年轻是应该奋斗的，不管是生活还是科研还是爱情，我希望自己一直保持对生活的热爱，拥抱世界。少一些对外界事物的敏感，其实世界很美好呀，独处时享受自己的时光，在一起时感受彼此的爱意，不需要一直证明，世界已经很爱我了。

-----

## 12/2 周一
从 **github** 上下载一个项目，根据项目创建一个新的虚拟环境。可通过命令：$\textbf{pip install -e .}$安装该环境里所有 python package，关于包的说明一般包含在文件 *setup.py* 里面。 
     
研究了 sam2 代码的使用：    
```
predictor.reset_state(inference_state)   #初始化状态   
predictor.add_new_points_or_box    #对指定帧添加点或框的 prompt    
predictor.propagate_in_video   #根据 prompt 对整个视频预测 
```    
/// 可以在中途添加预测，然后传播到整个视频。重新预测的话需要初始化状态。     

## 12/3 周二
**SAM2** ImageEncoder 由两部分组成：Hierarchical ViT[1] 和 remove CNN FpnNeck      
主要研究了 Hiera 的结构：使用最简单的层级式 *ViT* 结构结合 *MAE* 的代理任务实现强大的性能。    
- 大量使用窗口注意力实现高效计算，结合少量全局注意力实现全局特征交互。     
- 采用标准的多头注意力机制，注意力换成窗口注意力（在小窗口中计算注意力，窗口数量挪到 batch 维度）。    
- QKV 不是相等，是将上层输出经过线性层将 dim 扩充三倍，然后分离出 QKV，这样维持整体 shape 不变。   
- 实现层级转变时 Q转换成特征图使用$(2\times2)$池化，高宽减半。最后使用线性投影将 dim 变为原来两倍。
- 4个stage，patch_size=4 1024->256->128->64->32

![hiera结构](./image/hiera_arch.png)
[1] Ryali, Chaitanya, et al. "Hiera: A hierarchical vision transformer without the bells-and-whistles." *International Conference on Machine Learning(ICML).* PMLR, 2023.

## 12/4 周三
1. 首先是关于 `SAM2` 的 **image_encoder**
    - **hiera** 依靠 **attention block** 堆叠而成，每个 **block** 设置是否要`层级变换`和`窗口注意力`。   
    - **block** 的堆叠组成 **stage** ，每个 **stage** 包含一个层级，高宽减半，维度翻倍。    
    - 参数 **return_interm_layers** 决定返回每个 **stage** 的输出还是只返回最后的输出。**output** 是一个 list。

2. 然后抽空写了一下 `cover letter`。    
    又浪费了十天时间，在论文重投回去要写 cover letter ，修改作者也要在其中表明，并且写出作者贡献。好在最后靠`小婷`收获了一些写信的技巧。


## 12/6 周五
1. 知识科普      
    ```torch.rand```生成均值为 0.5 (0~1) 之间的随机数    
    ```torch.randn```生成均值为 0 方差为 1 的 (-1, 1) 之间的正态分布随机数    
    ```transforms.ToTensor```将数据归一到 (0~1)    

2. 关于 python 里的`方法`和`属性`   
    [**.size 是方法：**]需要加括号调用。    
    [**.shape 是属性：**]不需要加括号。   

## 12/7 周六     
1. 尝试本地部署大语言模型[**Llama**]      
    申请 llama3.2 权重下载，但是被拒绝了，半路夭折
2. 查看[**SAM**]项目时准备学习 `docker`    
    按照教程尝试安装，最后一步拉取镜像始终失败；已经换上阿里云docker镜像源，但是失败

### 服务器走本地代理
在 bash 里导入环境变量
```bash
export http_proxy="http://127.0.0.1:7897"
export https_proxy="http://127.0.0.1:7897"
```
测试    
```curl -x http://127.0.0.1:7897 http://www.google.com```

# 12月第一周总结
梳理一下，如果我要借助一项工具完成某项工作     
如果说我要魔改 SAM2 用于相同的任务，那么我要摸清他的结构知道怎么修改，但是这样难以做出优秀的工作。    
转念一想，好像也可以，不要做一些没有意义的小改动，而要引入创新性的新模块，有点跨领域的类似于生成模型。     
问题又出现了， SAM2 的任务太独特，很难搞。如果要有心意，我要放到别的任务上才有竞争性。   
坏了，越说越越乱，，看完可以吗，，，，真的责怪自己，即没有天赋也不够自律。

---

## 12/9 周一
1. **prompt_encoder**     
和 sam1 中基本没有区别

2. **mask_decoder**    
在 sam2 中 **mask_decoder** 多一个 pred_obj_scores token 预测遮挡分数      
图像反卷的时候可以选择加入两层前面 ImageEncoder 的高分辨率特征

```python
self.pred_obj_scores = pred_obj_scores
# 创建一个对应 token
self.obj_score_token = nn.Embedding(1, transformer_dim)
# 最后MLP将其维度转变为1
# 单幅图像预测一个遮挡，不用为每一mask预测遮挡分数
self.pred_obj_score_head = nn.Linear(transformer_dim, 1) 

return masks, iou_pred, sam_tokens_out, object_score_logits

masks.shape = (batch, 3, H, W)   # mask输出
iou_pre.shape = (batch, 3)   # iou分数预测
sam_tokens_out.shape = (batch, dim)   # 用于生成mask的token 取四个中的第一个 标记帧的对象指针
object_score_logits.shape = (batch, 1)   # 遮挡分数预测

# tokens: object_score + iou_pre + sam_tokens + masks (1, 1, 1, 3)
```

## 12/10 周二
1. **memory attention**    
    平平无奇的 *self-attention* 和 *cross-attention* 的交叉      
    但是使用旋转位置编码注意力：**RoPEAttention**    
    key 中空出一定数量的 token 不应用旋转位置编码

2. 一波三折，成功下载 **llama-1B** 权重，明天试试运行。     
    翻墙翻墙，还是翻墙，太难顶了。

`yaml` 写配置文件可以指定写入文件目标，对特定文件填入对应参数，用的是对应 `hydra` 包。层级式递进，结构简洁明了。

## 12/11 周三
今天最后一块拼图 memory encoder 快点看完吧，然后开始本地跑 llama

1. **memory encoder**     
    输入：masks and pix_feat    
    masks 下采样 dim=256    
    pix_feat 线性投影不改变高宽和维度 dim=256    
    然后经过两次融合网络 ConvNeXt Block 融合输出 并改变维度 dim=64   
    输出：{"vision_features": x, "vision_pos_enc": [pos]}   视觉特征和对应位置编码

2. **llama**    
    创建新的虚拟环境 llama3        
    跑通代码，但是仅限于固定例子，其实是道阻且长    
    还有自由对话，微调，等等


## 12/12 周四
### **cuda 安装**
#### 1. 先去官网找到对应版本 cuda: 12.4
```bash
wget https://developer.download.nvidia.com/compute/cuda/12.4.0/local_installers/cuda_12.4.0_550.54.14_linux.run
sudo sh cuda_12.4.0_550.54.14_linux.run
```
运行安装，如果有驱动，则取消安装驱动选项，仅仅安装cuda即可

#### 2. 配置 cuda 环境变量
```bash
vi ~/.bashrc
```
在文件末尾加入
```
export PATH=/usr/local/cuda-12.4/bin:$PATH
export LD_LIBRARY_PATH=/usr/local/cuda-12.4/lib64:$LD_LIBRARY_PATH
```
#### 3. 使更改生效
运行命令行
```bash
source ~/.bashrc
```
重启电脑，验证 cuda 是否成功安装
```bash
nvcc -V
```

不要让 vscode 帮下软件包，会损害环境    
下载 Jupyter-notebook 直接使用命令：
```pip install jupyter```

# 12月第二周总结
### 首先说明这一周的成果：
再一次一个个梳理了 **SAM2** 每个组件的代码，差不多有一个清晰的认知，但是问题在于最后的 **pipeline** 有点让人琢磨不透。     
大脑觉得这部分很难，自动不想学习，自动退缩，我自己都无语了😓。 

### 说明一下对未来工作的想象
1. 扩散模型很火     
    扩散生成模型有一步一步逐渐预测未来的属性；**DiT**和层级式的**UNet**，是否可以把**DiT**改成层级式结构重新应用。

2. 语言模型内容可以用到视觉模型      
    要追求新式的创新肯定要将其他领域的内容引用过来，讲好故事。prompt概念就是出自语言模型，可以寻找更多的语言模型和视觉模型的共同性。

3. 追求大一统，通用模型，世界模型     
    现在深度学习的发展已经进入非常深度的时期，普通的视觉，语言任务已经做的非常深入，指标也很高。大家都在寻找统一的智能体与世界交互，形成具身智能。未来发展的趋势，是否能在这里面寻找一丝契机。

扩散模型有时间步一步一步还原原始图像或者生成，语言模型有 *CoT* 的思想，这搬到视觉模型完美匹配。   
视觉模型也需要展现推理过程吗？等等，看过一篇论文就是扩散模型开放词汇的分割任务。

---
发现躺平是不可能的，现在的局势对我来说处境更加危险，全是未知，还有目前所面临的挑战，我真的不能松懈了。我得振作起来，要证明给别人看，向世人展示我的实力，迷茫与未知是暂时的，我要重新给自己树立信心，要明白自己的独一无二。     
人的本性是懒惰，懒惰使我的大脑变得迟钝。但是要想实现目标，攀登新的高峰，需要我们不断的[**思考**]，没有意识的形如行尸走肉，没有灵魂的躯壳，伟大的工作来自持续的思考与行动。    
可能某项工作对我来说太难了，不要急，我可以慢慢来，我拥有时间，设备，还有大脑。一项任务碰壁的时候，试着尝试其他任务，提升自己的自信心，多线发展，总有走得通的那条路。

## 12/16 周一
#### **Segment Everything Everywhere All at Once**
[ 2023/7/11 cite: 448 NeurIPS2023 ]    
差不多和 SAM 同期工作，内容也类似，但是有区别         
1. 设计 prompt 方案，将所有prompt投影到联合视觉语义空间，使其可以处理各种分割任务；   
2. 设计 *SEEM* 通用的交互式分割界面；    
3. 广泛的实验和可视化说明我们的模型在各种分割任务上有强大的性能。   
       
值得注意的是，本文包含全面的多模态 prompt ：词语；句子；点；框；涂鸦；相同语义图片。     
所有 prompt 投影到联合语义空间，可以理解语义，通过输入语义图片可以找出视频中相同语义物体。

评价是：没写代码，但是其实有GitHub。可以看出其实是一步一步完善的，本来可能只能部分任务，但是不断完善，可以处理不同的任务。文中对模型的描述不是特别清晰，但是实验超级多，这就是大组和公司合作吗。

## 12/17 周二
#### **Semantic-SAM: Segment and Recognize Anything at Any Granularity**
[ 2023/7/10 cite:142 ECCV2024]
具有语义意识和丰富粒度的通用分割模型，可点和框的交互式分割，也可语义分割，实例分割，全景分割。   
1. 为保证语义意识，将物体和部件解耦训练，物体学习语义信息，部件学习粒度信息。
2. 为保证粒度丰富性，采用匈牙利匹配的多对多训练策略。   

主体网络基于 Mask-DINO ，不支持文本 prompt 分割任务。文章的总体立意上不是特别吸引人：
1. 语义意识体现在有标签
2. 粒度体现在有多个mask

## 12/18 周三    
![扩散模型发展](./image/AutoEncoder.png)       
### **AE -> DAE -> VAE -> VQVAE -> 扩散模型**
#### **1. Autoencoder (AE)**
AE 是最基本的自编码器，通过一个编码器将输入数据压缩到低维的潜在空间（latent space），然后通过解码器重建原始输入。
#### **2. Denoising Autoencoder (DAE)**
DAE 在 AE 的基础上增加了抗噪能力。输入数据被添加噪声后传入模型，但模型的目标是重建无噪声的原始数据。
#### **3. Variational Autoencoder (VAE)**
- VAE 将 AE 拓展为概率模型，在潜在空间中引入了`概率分布假设`（通常是高斯分布）。
- 编码器输出的是潜在变量的均值和方差，通过随机采样生成潜在向量，从而增强模型的生成能力。
#### **4. Vector Quantized Variational Autoencoder (VQVAE)**
- VQVAE 是 VAE 的一种变体，使用离散潜在空间表示。编码器将数据映射到一个有限的离散向量集合（codebook），通过最近邻查找确定潜在表示。
- 引入了量化步骤，消除了潜在空间中的连续分布。
#### 5. 扩散模型
原始图像通过不断加入噪声得到各向同性的概率分布，这个过程称为 **forward diffusion** 。随后将噪声反向通过一个模型不断重建图像，所有的模型都是共享参数的。同时同一个模型的多步的重复使用也使得扩散模型成本比较贵，推理非常慢。后续改进：
- 预测图像可改为预测噪声，噪声均值和方差更好优化。
- 在模型里添加时间步信息，告诉模型到反向扩散的第几步。
- 原始的模型一般使用 **U-Net** 可以通过增强模型减少时间步和生成质量。
- **classifier guidance** 将每一步重建的特征通过一个分类头，使每一步图像尽量包含语义信息，增强生成稳定性。
- **guidance free** 方法生成一个条件结果一个无条件结果，使两个结果尽量接近，训练很贵但是效果很好。

目前更流行的做法是使用 *transformer* 模型，**DiT(diffusion transformer)**。


## 12/20 周五
#### **Segment Anything in High Quality**
[ 2023/10/23 cite:262 NeurIPS2023 ]        
改善 SAM 的 mask 分割精细度不够的问题     
1. 冻住原来 SAM 的参数，引入一个高质量的 input token
2. 将 ImageEncoder 中的前期特征与变换后的特征融合起来，点乘生成 mask

优势：可训练参数超级少，训练成本低；改动极小，保留了SAM 原始 的强大 zero-shot 能力。     
同时提出一个质量更好的44K个mask的数据集，提升模型性能。8x3090训练4个小时。

评价是：想法非常超级简单，但是其中融合前期图像特征的想法被 SAM2 中借鉴，说明确实有效。
